# ElasticNet

In [1]:
path=  'C:/Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/'
#import sys
#sys.path.append(path) 
path_=  'C:/Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Model/'
import sys
sys.path.append(path_) 

In [2]:
from sklearn.linear_model import ElasticNet
from base_regressor import BaseRegressor
from Plotter import Plotter
from skopt.space import Real, Categorical, Integer

class ElasticNetRegressor(BaseRegressor):
    def __init__(self, save_path=None, scaler=None, params=None, params_space=None, fit_params_search=None, model_params_search=None, fit_params_train=None, model_params_train=None, name_model="ElasticNet"):
        super().__init__(save_path, scaler, params, params_space, fit_params_search, model_params_search, fit_params_train, model_params_train, name_model)

        self.model_ml = ElasticNet
        if params is None:
            self.params = {
                'alpha': 0.2,
                'l1_ratio': 0.5,  # Proporción de L1 en la regularización
                'max_iter': 10000,
                # 'tol': 0.001
            }
        if params_space is None:
            self.params_space = {
                'alpha': Real(0.001, 10.0, prior='log-uniform'),
                'l1_ratio': Real(0.0, 1.0),  # Rango de 0 a 1 para la proporción de L1
                'max_iter': Integer(100, 10000),
                'tol': Real(1e-5, 1e-2, prior='log-uniform')
            }


c:\python_env\brain_age_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Clasificador XGBoost

In [3]:
from base_classifier import BaseClassifier
from Plotter import Plotter
from skopt.space import Real, Categorical, Integer
import xgboost as xgb

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold


class XGBoostClassifier(BaseClassifier):
    def __init__(self, save_path=None, scaler=None, params=None, params_space=None, fit_params_search=None, model_params_search=None, fit_params_train=None, model_params_train=None, name_model="Lasso"):
        super().__init__(save_path, scaler, params, params_space, fit_params_search, model_params_search, fit_params_train, model_params_train, name_model)

        self.model_ml = xgb.XGBClassifier
        if params is None:
            self.params = {
            'objective': 'binary:logistic',  # Clasificación binaria
            'eval_metric': 'logloss',       # Métrica de evaluación
            'learning_rate': 0.1,           # Tasa de aprendizaje
            'max_depth': 6,                 # Profundidad máxima del árbol
            'subsample': 0.8,               # Proporción de muestras utilizadas por árbol
            'colsample_bytree': 0.8,        # Proporción de características utilizadas por árbol
            'reg_alpha': 0.1,               # Regularización L1
            'reg_lambda': 1.0,              # Regularización L2
            'gamma': 0.0,                   # Mínima reducción de pérdida para crear una división
            'min_child_weight': 1,          # Mínimo peso de la suma de los hijos para una división
            'n_estimators': 100,            # Número de árboles en el modelo
            'scale_pos_weight': 1.0         # Para datos desbalanceados
        }
        if params_space is None:
            self.params_space = {
            'objective': ['binary:logistic'],
            'eval_metric': ['logloss', 'error', 'auc', 'aucpr'],
            'learning_rate': Real(0.01, 0.3),
            'max_depth': Integer(3, 10),
            'subsample': Real(0.5, 1.0),
            'colsample_bytree': Real(0.5, 1.0),
            'reg_alpha': Real(0, 1.0),
            'reg_lambda': Real(0, 1.0),
            'gamma': Real(0.0, 10.0),
            'min_child_weight': Integer(1, 10),
            'scale_pos_weight': Real(1.0, 10.0),
            'n_estimators': Integer(100, 500),  # Número de árboles a probar
        }

# Instancia de Modelos

In [4]:
model = ElasticNetRegressor()
model_cls = XGBoostClassifier()
Plotters = Plotter()

nameset ='ElasticNet'

# Data

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import pickle

path_metric = f'{path}Multiple_Brain_Age_Clocks/pre_data/'


file_path_CN = f'{path_metric}CN_combined.xlsx'
file_path_AD = f'{path_metric}AD_combined.xlsx'
file_path_FTD = f'{path_metric}FTD_combined.xlsx'
file_path_MCI = f'{path_metric}MCI_combined.xlsx'

df_CN = pd.read_excel(file_path_CN)
df_AD = pd.read_excel(file_path_AD)
df_FTD = pd.read_excel(file_path_FTD)
df_MCI = pd.read_excel(file_path_MCI)

df_CN_filtrado = df_CN[(df_CN['Age'] >=50) & (df_CN['Age'] <= 90)].reset_index(drop=True)
df_AD_filtrado = df_AD[(df_AD['Age'] >= 50) & (df_AD['Age'] <= 90)].reset_index(drop=True)
df_FTD_filtrado = df_FTD[(df_FTD['Age'] >= 50) & (df_FTD['Age'] <= 90)].reset_index(drop=True)
df_MCI_filtrado = df_MCI[(df_MCI['Age'] >= 50) & (df_MCI['Age'] <= 90)].reset_index(drop=True)


In [6]:
metrics_freq = df_CN.iloc[:, 62:280].columns.tolist()
metrics_aperiodic = df_CN.iloc[:, 1:31].columns.tolist()
metrics_complex = df_CN.iloc[:, 43:52].columns.tolist()


In [7]:
#CN
X_CN_freq = df_CN_filtrado[metrics_freq]
X_CN_aperiodic= df_CN_filtrado[metrics_aperiodic]
X_CN_complex= df_CN_filtrado[metrics_complex]
X_CN = pd.concat([X_CN_freq, X_CN_aperiodic, X_CN_complex], axis=1)
y_CN = df_CN_filtrado["Age"]
ID_CN = df_CN_filtrado["ID-unique"]

#Esto es para limpiar los valores nulos
df_concatenado_CN_ = pd.concat([X_CN, y_CN, ID_CN], axis=1, ignore_index=False).dropna(axis=0) 

X_CN =df_concatenado_CN_.iloc[:, :-2]
X_CN_freq = df_concatenado_CN_[metrics_freq]
X_CN_aperiodic= df_concatenado_CN_[metrics_aperiodic]
X_CN_complex= df_concatenado_CN_[metrics_complex]
y_CN= df_concatenado_CN_.iloc[:, -2]   # Labels (Age)
ID_CN = df_concatenado_CN_.iloc[:, -1] 

#AD
X_AD_freq = df_AD_filtrado[metrics_freq]
X_AD_aperiodic= df_AD_filtrado[metrics_aperiodic]
X_AD_complex= df_AD_filtrado[metrics_complex]
X_AD = pd.concat([X_AD_freq, X_AD_aperiodic, X_AD_complex], axis=1)
y_AD = df_AD_filtrado["Age"]
ID_AD = df_AD_filtrado["ID-unique"]

df_concatenado_AD_ = pd.concat([X_AD, y_AD, ID_AD], axis=1, ignore_index=False).dropna(axis=0) 

X_AD =df_concatenado_AD_.iloc[:, :-2]
X_AD_freq = df_concatenado_AD_[metrics_freq]
X_AD_aperiodic= df_concatenado_AD_[metrics_aperiodic]
X_AD_complex= df_concatenado_AD_[metrics_complex]
y_AD= df_concatenado_AD_.iloc[:, -2]   # Labels (Age)
ID_AD = df_concatenado_AD_.iloc[:, -1] 

#FTD
X_FTD_freq = df_FTD_filtrado[metrics_freq]
X_FTD_aperiodic = df_FTD_filtrado[metrics_aperiodic]
X_FTD_complex = df_FTD_filtrado[metrics_complex]
X_FTD = pd.concat([X_FTD_freq, X_FTD_aperiodic, X_FTD_complex], axis=1)
y_FTD = df_FTD_filtrado["Age"]
ID_FTD = df_FTD_filtrado["ID-unique"]

df_concatenado_FTD_ = pd.concat([X_FTD, y_FTD, ID_FTD], axis=1, ignore_index=False).dropna(axis=0) 

X_FTD_freq = df_concatenado_FTD_[metrics_freq]
X_FTD_aperiodic = df_concatenado_FTD_[metrics_aperiodic]
X_FTD_complex = df_concatenado_FTD_[metrics_complex]
X_FTD =df_concatenado_FTD_.iloc[:, :-2]
y_FTD= df_concatenado_FTD_.iloc[:, -2]   # Labels (Age)
ID_FTD = df_concatenado_FTD_.iloc[:, -1] 

#MCI
X_MCI_freq = df_MCI_filtrado[metrics_freq]
X_MCI_aperiodic = df_MCI_filtrado[metrics_aperiodic]
X_MCI_complex = df_MCI_filtrado[metrics_complex]
X_MCI = pd.concat([X_MCI_freq, X_MCI_aperiodic, X_MCI_complex], axis=1)
y_MCI = df_MCI_filtrado["Age"]
ID_MCI = df_MCI_filtrado["ID-unique"]

df_concatenado_MCI_ = pd.concat([X_MCI, y_MCI, ID_MCI], axis=1, ignore_index=False).dropna(axis=0) 
X_MCI_freq = df_concatenado_MCI_[metrics_freq]
X_MCI_aperiodic = df_concatenado_MCI_[metrics_aperiodic]
X_MCI_complex = df_concatenado_MCI_[metrics_complex]
X_MCI =df_concatenado_MCI_.iloc[:, :-2]
y_MCI= df_concatenado_MCI_.iloc[:, -2]   # Labels (Age)
ID_MCI = df_concatenado_MCI_.iloc[:, -1] 


In [9]:
from sklearn.preprocessing import StandardScaler

# Inicializamos el escalador
scaler = StandardScaler()

# Escalamos cada conjunto
X_CN_freq_scaled = pd.DataFrame(scaler.fit_transform(X_CN_freq), columns=X_CN_freq.columns, index=X_CN_freq.index)
X_CN_aperiodic_scaled = pd.DataFrame(scaler.fit_transform(X_CN_aperiodic), columns=X_CN_aperiodic.columns, index=X_CN_aperiodic.index)
X_CN_complex_scaled = pd.DataFrame(scaler.fit_transform(X_CN_complex), columns=X_CN_complex.columns, index=X_CN_complex.index)
X_CN_scaled = pd.DataFrame(scaler.fit_transform(X_CN), columns=X_CN.columns, index=X_CN.index)

# Análisis por Métricas

In [ ]:
nombres_regiones = [
    'OCC_left', 'OCC_right', 'HPC_left', 'HPC_right', 
    'PARIET_left', 'PARIET_right', 'CING_left', 'CING_right', 
    'ORB_left', 'ORB_right', 'INS_left', 'INS_right', 
    'IFG_left', 'IFG_right', 'SFG_left', 'SFG_right', 
    'MFG_left', 'MFG_right', 'TEMP_left','TEMP_right'
]

columnas_por_region_global = {}

for nombre_conjunto, df_conjunto in scaled_data_CN.items():
    columnas_por_region = {}
    for region in nombres_regiones:  
        columnas_por_region[region] = [col for col in df_conjunto.columns if region in col]
    
    columnas_por_region_global[nombre_conjunto] = columnas_por_region


In [ ]:
for region in nombres_regiones:  
    columnas_por_region[region] = [col for col in df_conjunto.columns if region in col]

In [19]:
columnas_por_region_global['complex']

{'OCC_left': ['SSV_OCC_left'],
 'OCC_right': ['SSV_OCC_right'],
 'HPC_left': [],
 'HPC_right': [],
 'PARIET_left': [],
 'PARIET_right': [],
 'CING_left': [],
 'CING_right': [],
 'ORB_left': [],
 'ORB_right': [],
 'INS_left': [],
 'INS_right': [],
 'IFG_left': ['SSV_IFG_left'],
 'IFG_right': ['SSV_IFG_right'],
 'SFG_left': [],
 'SFG_right': [],
 'MFG_left': [],
 'MFG_right': [],
 'TEMP_left': ['SSV_TEMP_left'],
 'TEMP_right': ['SSV_TEMP_right']}

# Esto ya no lo hago

In [11]:
grupos = ['freq', 'aperiodic', 'complex', 'freq+aperiodic', 'freq+complex', 'aperiodic+complex', 'freq+aperiodic+complex']

# Busqueda hiperparametros

In [21]:
columnas_por_region_global.keys()

dict_keys(['freq', 'aperiodic', 'complex', 'freq+aperiodic', 'freq+complex', 'aperiodic+complex', 'freq+aperiodic+complex'])

In [ ]:
for nombre_conjunto, df_conjunto in scaled_data_CN.items():
    

In [ ]:
# Diccionario para almacenar los modelos y parámetros óptimos por conjunto
resultados_modelos = {}

# Iterar sobre los conjuntos escalados
for nombre_conjunto, df_conjunto in scaled_data_CN.items():
    print(f"Procesando conjunto: {nombre_conjunto}")
    
    # Crear listas para almacenar los modelos y los mejores parámetros
    #opt_models = []
    best_params = []
    opt_models = {}
    
    # Iterar sobre las regiones
    for region in nombres_regiones:
        # Nombre de las columnas asociadas a la región
        columnas_region = columnas_por_region_global[nombre_conjunto][region]

        if not columnas_region:
            continue
        

        else: 
            # Aplicar el modelo para las columnas de la región
            opt_model, best_params_region = model.search_best_model(
                X=df_conjunto[columnas_region], 
                y=y_CN,  # Asegúrate de que y_CN sea aplicable a todos los conjuntos
                n_iter_=50, 
                scoring_metric='r2'
            )

            opt_models[region] = opt_model
        
        # Agregar los resultados a las listas
        #opt_models.append(opt_model)
        #best_params.append(best_params_region)
        
        #print(f"Modelo y parámetros para región {region} en conjunto {nombre_conjunto} procesados.")
    resultados_modelos[nombre_conjunto] = opt_models
    # Guardar los resultados en el diccionario global
    #resultados_modelos[nombre_conjunto] = {
    #    "modelos": opt_models,
    #    "parametros": best_params
    #}

# Mostrar un resumen
#print("Procesamiento completado.")
#for conjunto, resultados in resultados_modelos.items():
#    print(f"Se ajustaron {len(resultados['modelos'])} modelos para el conjunto {conjunto}.")


In [43]:
resultados_modelos.keys()

dict_keys(['freq', 'aperiodic', 'complex', 'freq+aperiodic', 'freq+complex', 'aperiodic+complex', 'freq+aperiodic+complex'])

In [46]:

# Guardar los resultados en un archivo Pickle
pickle_filename = f"results_model/resultados_bayes.pkl"
with open(pickle_filename, 'wb') as f:
    pickle.dump(resultados_modelos, f)

print(f"Resultados guardados en {pickle_filename}")

Resultados guardados en results_model/resultados_bayes.pkl


In [13]:
# Cargar los resultados
with open('results_model/resultados_bayes.pkl', 'rb') as file_labels:
    resultados_modelos = pickle.load(file_labels)

In [14]:
for nombre_conjunto, df_conjunto in resultados_modelos.items():
    print(nombre_conjunto)

freq
aperiodic
complex
freq+aperiodic
freq+complex
aperiodic+complex
freq+aperiodic+complex


In [47]:
resultados_modelos

{'freq': {'OCC_left': BayesSearchCV(cv=KFold(n_splits=10, random_state=126, shuffle=True),
                estimator=ElasticNet(), fit_params={}, n_jobs=-1, random_state=42,
                scoring='r2',
                search_spaces={'alpha': Real(low=0.001, high=10.0, prior='log-uniform', transform='normalize'),
                               'l1_ratio': Real(low=0.0, high=1.0, prior='uniform', transform='normalize'),
                               'max_iter': Integer(low=100, high=10000, prior='uniform', transform='normalize'),
                               'tol': Real(low=1e-05, high=0.01, prior='log-uniform', transform='normalize')},
                verbose=1),
  'OCC_right': BayesSearchCV(cv=KFold(n_splits=10, random_state=126, shuffle=True),
                estimator=ElasticNet(), fit_params={}, n_jobs=-1, random_state=42,
                scoring='r2',
                search_spaces={'alpha': Real(low=0.001, high=10.0, prior='log-uniform', transform='normalize'),
               

In [49]:
for region in nombres_regiones:
    print(region)

OCC_left
OCC_right
HPC_left
HPC_right
PARIET_left
PARIET_right
CING_left
CING_right
ORB_left
ORB_right
INS_left
INS_right
IFG_left
IFG_right
SFG_left
SFG_right
MFG_left
MFG_right
TEMP_left
TEMP_right


In [ ]:
resultados_modelos['aperiodic']['ORB_left']

In [59]:
best_params_modelos={}

# Diccionario para almacenar los mejores hiperparámetros por región
best_params_all_regions = {}

for nombre_conjunto, df_conjunto in resultados_modelos.items():

    # Iterar sobre las regiones y los modelos optimizados
    for region in nombres_regiones:
        # Obtener los mejores hiperparámetros para el modelo de la región
        try:
            best_params_ = model.best_hyper(num_best=1, opt_model=resultados_modelos[nombre_conjunto][region], num_max=50)
            
            # Guardar los resultados en el diccionario
            best_params_all_regions[region] = best_params_
        except:
            print('La region no existe')

    best_params_modelos[nombre_conjunto]=best_params_all_regions
        # print(f"Mejores hiperparámetros procesados para {region}.")  # Seguimiento del progreso

    # Mostrar un resumen
    #p rint("Procesamiento completado.")


La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe
La region no existe


In [62]:
best_params_modelos

{'freq': {'OCC_left': [{'alpha': 0.2838853257182737,
    'l1_ratio': 1.0,
    'max_iter': 10000,
    'tol': 1e-05}],
  'OCC_right': [{'alpha': 0.3290937475037383,
    'l1_ratio': 1.0,
    'max_iter': 100,
    'tol': 1e-05}],
  'HPC_left': [{'alpha': 1.3448256268883396,
    'l1_ratio': 0.04566627045993015,
    'max_iter': 10000,
    'tol': 1e-05}],
  'HPC_right': [{'alpha': 0.3485145657429254,
    'l1_ratio': 0.9838866167627093,
    'max_iter': 10000,
    'tol': 0.01}],
  'PARIET_left': [{'alpha': 1.7351135073317683,
    'l1_ratio': 0.0,
    'max_iter': 10000,
    'tol': 0.01}],
  'PARIET_right': [{'alpha': 2.6807908068911344,
    'l1_ratio': 0.0,
    'max_iter': 10000,
    'tol': 0.01}],
  'CING_left': [{'alpha': 0.16167236370100946,
    'l1_ratio': 0.9601536689960662,
    'max_iter': 100,
    'tol': 0.01}],
  'CING_right': [{'alpha': 0.16893544819212186,
    'l1_ratio': 1.0,
    'max_iter': 10000,
    'tol': 0.01}],
  'ORB_left': [{'alpha': 0.33031480069992114,
    'l1_ratio': 0.0,
  

In [60]:
best_params_modelos['freq']

{'OCC_left': [{'alpha': 0.2838853257182737,
   'l1_ratio': 1.0,
   'max_iter': 10000,
   'tol': 1e-05}],
 'OCC_right': [{'alpha': 0.3290937475037383,
   'l1_ratio': 1.0,
   'max_iter': 100,
   'tol': 1e-05}],
 'HPC_left': [{'alpha': 1.3448256268883396,
   'l1_ratio': 0.04566627045993015,
   'max_iter': 10000,
   'tol': 1e-05}],
 'HPC_right': [{'alpha': 0.3485145657429254,
   'l1_ratio': 0.9838866167627093,
   'max_iter': 10000,
   'tol': 0.01}],
 'PARIET_left': [{'alpha': 1.7351135073317683,
   'l1_ratio': 0.0,
   'max_iter': 10000,
   'tol': 0.01}],
 'PARIET_right': [{'alpha': 2.6807908068911344,
   'l1_ratio': 0.0,
   'max_iter': 10000,
   'tol': 0.01}],
 'CING_left': [{'alpha': 0.16167236370100946,
   'l1_ratio': 0.9601536689960662,
   'max_iter': 100,
   'tol': 0.01}],
 'CING_right': [{'alpha': 0.16893544819212186,
   'l1_ratio': 1.0,
   'max_iter': 10000,
   'tol': 0.01}],
 'ORB_left': [{'alpha': 0.33031480069992114,
   'l1_ratio': 0.0,
   'max_iter': 2104,
   'tol': 1e-05}],
 'OR

In [87]:
with open('best_params', 'wb') as f:
    pickle.dump(best_params_modelos, f)

In [ ]:
with open('best_params.pkl', 'rb') as file_labels:
    best_params_modelos = pickle.load(file_labels)

In [ ]:
resultados_modelos[nombre_conjunto][region]

In [ ]:
results_CN={}

results_by_region = {}

for nombre_conjunto, params_modelo in best_params_modelos.items():
    for region in nombres_regiones:
        columnas_region = columnas_por_region[region]
        if not columnas_region:
            continue

        # Obtener los mejores parámetros para la región
        best_params_region = best_params_modelos[nombre_conjunto][region]

        # Entrenar el modelo con `trainer_2`
        labels_train, labels_test, model_, folds_train, folds_test = model.trainer_2(
            X=X_CN[columnas_region],
            y=y_CN,
            ID=ID_CN,
            n_splits=10,
            n_iterations=20,
            params_=best_params_region[0]
        )

        # Guardar los resultados en un diccionario
        results_by_region[region] = {
            "labels_train": labels_train,
            "labels_test": labels_test,
            "model": model_,
            "folds_train": folds_train,
            "folds_test": folds_test,
        }

        print(f"Modelo entrenado para la región {region}.")  # Seguimiento del progreso

            
        #print(f"Modelo entrenado para la región {region}.")  # Seguimiento del progreso

    results_CN[nombre_conjunto]=results_by_region
    # Mostrar un resumen
    print("Entrenamiento completado para todas las regiones.")


In [80]:
results_CN['freq']['OCC_left']['model']

{'model': [ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=0.2838853257182737, l1_ratio=1.0, max_iter=10000, tol=1e-05),
  ElasticNet(alpha=

In [77]:
with open('results_CN', 'wb') as f:
    pickle.dump(results_CN, f)


In [66]:
columnas_por_region_global['aperiodic']['ORB_left']

[]

In [81]:
results_AD={}
results_FTD={}
results_MCI={}



# Diccionarios para almacenar resultados por región y paciente
#results_labels_test = {'AD': {}, 'FTD': {}, 'MCI': {}}
#results_folds_test = {'AD': {}, 'FTD': {}, 'MCI': {}}

for nombre_conjunto in scaled_data_CN.keys():

    results_labels_test_AD = {}
    results_labels_test_FTD = {}
    results_labels_test_MCI = {}

    # Iterar sobre las regiones
    for region in nombres_regiones:
        columnas_region = columnas_por_region[region]
        if not columnas_region:
            continue

        else:
            modelo_region = results_CN[nombre_conjunto][region]['model']
        
            # Test para pacientes AD
            results_labels_test_AD[region], results_labels_test_AD[region] = model.test(
                X=X_AD[columnas_region],
                y=y_AD,
                ID=ID_AD,
                n_splits=10,
                n_iterations=20,
                result_model=modelo_region
            )
            
            # Test para pacientes FTD
            results_labels_test_FTD[region], results_labels_test_FTD[region] = model.test(
                X=X_FTD[columnas_region],
                y=y_FTD,
                ID=ID_FTD,
                n_splits=10,
                n_iterations=20,
                result_model=modelo_region
            )
            
            # Test para pacientes MCI
            results_labels_test_MCI[region], results_labels_test_MCI[region] = model.test(
                X=X_MCI[columnas_region],
                y=y_MCI,
                ID=ID_MCI,
                n_splits=10,
                n_iterations=20,
                result_model=modelo_region
            )

    results_AD[nombre_conjunto]=results_labels_test_AD
    results_FTD[nombre_conjunto]=results_labels_test_FTD
    results_MCI[nombre_conjunto]=results_labels_test_MCI
            
    #print(f"Pruebas realizadas para la región {region}.")  # Seguimiento del progreso

# Mostrar un resumen
print("Pruebas completadas para todas las regiones y pacientes.")


C:\Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Model\base_regressor.py:469: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_labels_df_test = pd.concat([results_labels_df_test, temp_pat_df], ignore_index=True)
C:\Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Model\base_regressor.py:469: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_labels_df_test = pd.concat([results_labels_df_test, temp_pat_df], ign

Pruebas completadas para todas las regiones y pacientes.
